<a href="https://colab.research.google.com/github/ArkanDash/Advanced-RVC-Inference/blob/master/Advanced-RVC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced RVC Inference

A modular Retrieval-based Voice Conversion framework with Gradio UI, training capabilities, and audio processing tools

[Discord](https://discord.gg/hvmsukmBHE) - [Github](https://github.com/ArkanDash/Advanced-RVC-Inference)

In [ ]:
# @title Mount Drive
from google.colab import drive
from google.colab._message import MessageError

try:
  drive.mount("/content/drive")
except MessageError:
  print("❌ Failed to mount drive")

In [ ]:
# @title Setup runtime environment
# @markdown ### Choose installation method:
install_method = "local"  # @param ["local", "git+uv", "git+pip"]
from IPython.display import clear_output
import os

# Define paths first
LOGS_PATH = "/content/Advanced-RVC-Inference/advanced_rvc_inference/assets/logs"
BACKUPS_PATH = f"/content/drive/MyDrive/RVCBackup"

# Install system dependencies for audio processing
!apt-get -y install libportaudio2 ffmpeg git -qq > /dev/null 2>&1

# Install uv for faster package management
!pip install uv -q

if install_method == 'local':
    # Clone from git and install locally (Recommended for development)
    if os.path.exists("Advanced-RVC-Inference"):
        %cd Advanced-RVC-Inference
        !git pull origin master
    else:
        !git clone https://github.com/ArkanDash/Advanced-RVC-Inference.git
        %cd Advanced-RVC-Inference

    # Install from requirements.txt
    !uv pip install -r requirements.txt --system -q

    # Install package in editable mode
    !uv pip install -e . --system -q

elif install_method == 'git+uv':
    # Install latest version from GitHub with uv (fastest)
    !uv pip install git+https://github.com/ArkanDash/Advanced-RVC-Inference.git -q

elif install_method == 'git+pip':
    # Install latest version from GitHub with pip
    !pip install git+https://github.com/ArkanDash/Advanced-RVC-Inference.git -q

clear_output()
print("Finished installing Advanced RVC Inference!")
print("Use the cell below to start the server.")

In [ ]:
# @title Sync with Google Drive
# @markdown 💾 Run this cell to automatically Save/Load models from your mounted drive
# @markdown This will merge and link your `RVCBackup` folder from gdrive to this notebook
from IPython.display import display, clear_output
from pathlib import Path
import shutil
import subprocess
from datetime import datetime
import json

non_bak_folders = ["mute", "reference", "zips", "mute_spin", "mute_spin-v2"]
non_bak_path = "/tmp/rvc_logs"

def get_size(path: Path):
    """Get human readable size of directory"""
    try:
        total = 0
        for root, dirs, files in os.walk(path):
            for f in files:
                fp = os.path.join(root, f)
                if os.path.exists(fp):
                    total += os.path.getsize(fp)
        # Convert to human readable format
        for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
            if total < 1024.0:
                return f"{total:.1f} {unit}"
            total /= 1024.0
        return f"{total:.1f} PB"
    except:
        return "Unknown size"

def backup_model_to_drive(model_name, logs_path, backup_path):
    """Backup individual model to Google Drive"""
    try:
        model_path = Path(logs_path) / model_name
        backup_model_path = Path(backup_path) / model_name
        
        if model_path.exists():
            if backup_model_path.exists():
                shutil.rmtree(backup_model_path)
            
            # Copy model to backup
            shutil.copytree(model_path, backup_model_path)
            print(f"✓ Backed up {model_name}")
            return True
    except Exception as e:
        print(f"✗ Failed to backup {model_name}: {str(e)}")
    return False

def restore_model_from_drive(model_name, logs_path, backup_path):
    """Restore individual model from Google Drive"""
    try:
        model_path = Path(logs_path) / model_name
        backup_model_path = Path(backup_path) / model_name
        
        if backup_model_path.exists():
            if model_path.exists():
                shutil.rmtree(model_path)
            
            # Copy model from backup
            shutil.copytree(backup_model_path, model_path)
            print(f"✓ Restored {model_name}")
            return True
    except Exception as e:
        print(f"✗ Failed to restore {model_name}: {str(e)}")
    return False

def setup_realtime_backup(logs_path, backup_path):
    """Setup realtime backup monitoring"""
    try:
        # Create backup directory if it doesn't exist
        Path(backup_path).mkdir(parents=True, exist_ok=True)
        
        # Create a simple backup configuration file
        backup_config = {
            "last_backup": datetime.now().isoformat(),
            "backup_path": backup_path,
            "logs_path": logs_path,
            "auto_backup": True
        }
        
        config_file = Path(backup_path) / "backup_config.json"
        with open(config_file, 'w') as f:
            json.dump(backup_config, f, indent=2)
        
        print("✓ Realtime backup system initialized")
        return True
    except Exception as e:
        print(f"✗ Failed to setup realtime backup: {str(e)}")
        return False

def sync_models(logs_path, backup_path, force_restore=False):
    """Main sync function"""
    try:
        logs_path = Path(logs_path)
        backup_path = Path(backup_path)
        
        # Create directories if they don't exist
        logs_path.mkdir(parents=True, exist_ok=True)
        backup_path.mkdir(parents=True, exist_ok=True)
        
        # Setup realtime backup
        setup_realtime_backup(logs_path, backup_path)
        
        if force_restore:
            # Restore all models from backup
            print("Restoring all models from backup...")
            for item in backup_path.iterdir():
                if item.is_dir() and item.name not in non_bak_folders:
                    restore_model_from_drive(item.name, logs_path, backup_path)
        else:
            # Backup all models to drive
            print("Backing up models to Google Drive...")
            for item in logs_path.iterdir():
                if item.is_dir() and item.name not in non_bak_folders:
                    backup_model_to_drive(item.name, logs_path, backup_path)
        
        print("\n✓ Sync completed successfully!")
        
        # Show sync stats
        print("\nSync Statistics:")
        print(f"Logs Path: {logs_path}")
        print(f"Backup Path: {backup_path}")
        print(f"Non-backed up folders: {', '.join(non_bak_folders)}")
        
        return True
    
    except Exception as e:
        print(f"✗ Sync failed: {str(e)}")
        return False

# Run sync
if Path("/content/drive").is_mount():
    print("Google Drive mounted. Starting sync...\n")
    
    # Ask user what they want to do
    print("Choose sync option:")
    print("1. Backup local models to Google Drive")
    print("2. Restore models from Google Drive")
    
    try:
        choice = input("Enter choice (1 or 2, default is 1): ").strip()
        force_restore = (choice == "2")
    except:
        force_restore = False
    
    sync_models(LOGS_PATH, BACKUPS_PATH, force_restore)
    
    print("\nNote: Models are now being monitored for realtime backup.")
    print("Any changes to models will be automatically synced.")

else:
    print("❌ Google Drive is not mounted.")
    print("Please mount your Google Drive first and re-run this cell.")

In [ ]:
# @title **Start server**
# @markdown ### Choose a sharing method:
method = "gradio"  # @param ["gradio", "localtunnel", "ngrok"]
ngrok_token = ""  # @param {type:"string"}
from IPython.display import clear_output
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

# Additional environment variables for better performance
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

# Change to project directory if using local installation
if install_method == 'local':
    repo_dir = "Advanced-RVC-Inference"
    # Check if we're already in the repo directory
    if os.path.basename(os.getcwd()) != repo_dir:
        if os.path.exists(repo_dir):
            %cd $repo_dir
            !git pull -q

clear_output()
print("Starting Advanced RVC Inference Server...\n")

if method == 'gradio':
    # Start with share option
    print("Starting Gradio server with public link...")
    !rvc-gui --share

elif method == 'localtunnel':
    !pip install localtunnel -q
    clear_output()
    print("Starting server with localtunnel...")
    print("\nPublic IP: $(curl -s ifconfig.me)")
    print("\nStarting localtunnel in background...")
    # Start localtunnel and RVC server in parallel
    !lt --port 7869 --subdomain rvc-advanced &> /tmp/lt.log &
    !sleep 5
    !rvc-gui --listen --server-name 0.0.0.0

elif method == 'ngrok':
    !pip install pyngrok -q
    from pyngrok import ngrok, conf
    
    # Kill existing ngrok tunnels
    ngrok.kill()
    
    if ngrok_token:
        ngrok.set_auth_token(ngrok_token)
    
    clear_output()
    print("Starting server with ngrok...")
    
    # Create ngrok tunnel
    public_url = ngrok.connect(7869)
    print(f"\nNgrok URL: {public_url}")
    
    # Start RVC server
    !rvc-gui --listen --server-name 0.0.0.0 --server-port 7869

# Real-time monitoring and backup service (runs in background)
print("\n\nStarting real-time backup service...")
print("This service will monitor for model changes and auto-backup.")

# Create a simple monitoring script
monitor_script = '''
#!/usr/bin/env python3
import os
import time
import json
from pathlib import Path
from datetime import datetime
import shutil

LOGS_PATH = "/content/Advanced-RVC-Inference/advanced_rvc_inference/assets/logs"
BACKUPS_PATH = "/content/drive/MyDrive/RVCBackup"
CONFIG_FILE = os.path.join(BACKUPS_PATH, "backup_config.json")

def load_config():
    if os.path.exists(CONFIG_FILE):
        with open(CONFIG_FILE, 'r') as f:
            return json.load(f)
    return {}

def save_config(config):
    with open(CONFIG_FILE, 'w') as f:
        json.dump(config, f, indent=2)

def backup_modified_models():
    config = load_config()
    if not config.get('auto_backup', True):
        return
    
    logs_path = Path(LOGS_PATH)
    backup_path = Path(BACKUPS_PATH)
    
    if not logs_path.exists() or not backup_path.exists():
        return
    
    for model_dir in logs_path.iterdir():
        if model_dir.is_dir() and model_dir.name not in ['mute', 'reference', 'zips']:
            backup_dir = backup_path / model_dir.name
            
            # Check if model was modified
            if not backup_dir.exists() or \
               model_dir.stat().st_mtime > backup_dir.stat().st_mtime:
                try:
                    if backup_dir.exists():
                        shutil.rmtree(backup_dir)
                    shutil.copytree(model_dir, backup_dir)
                    print(f"[Auto-Backup] Backed up {model_dir.name}")
                except Exception as e:
                    print(f"[Auto-Backup] Failed to backup {model_dir.name}: {e}")
    
    config['last_backup'] = datetime.now().isoformat()
    save_config(config)

if __name__ == "__main__":
    print("Real-time backup monitor started...")
    print(f"Monitoring: {LOGS_PATH}")
    print(f"Backup to: {BACKUPS_PATH}")
    
    while True:
        try:
            backup_modified_models()
            time.sleep(60)  # Check every minute
        except KeyboardInterrupt:
            print("\nBackup monitor stopped.")
            break
        except Exception as e:
            print(f"Monitor error: {e}")
            time.sleep(60)
'''

# Write and start the monitor script
with open('/tmp/backup_monitor.py', 'w') as f:
    f.write(monitor_script)

# Start the monitor in the background
!python /tmp/backup_monitor.py > /tmp/backup_monitor.log 2>&1 &

print("\n✅ Backup service started. Check /tmp/backup_monitor.log for details.")